In [1]:
import pandas as pd
import numpy as np
from numpy import nan

In [2]:
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
songs_extra = pd.read_csv('song_extra_info.csv')
sample = pd.read_csv('sample_submission.csv')

Скопируем функции из первой части, чтобы убрать наны и разделить жанры

In [3]:
def nan(df):
  for col in df.columns:
    print(df[col].isna().sum()) 
nan(songs_extra)
songs_extra['isrc'] = songs_extra['isrc'].fillna('Unknown')
songs_extra['name'] = songs_extra['name'].fillna('Unknown')
songs_extra.head()

0
2
136548


,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [4]:
songs = pd.read_csv('songs.csv')
songs['genre_ids'] = songs['genre_ids'].fillna(0)
songs['composer'] = songs['composer'].fillna('Unknown')
songs['lyricist'] = songs['lyricist'].fillna('Unknown')
songs['language'] = songs['language'].fillna(0)
def genre_dev(genre, ind):
    splitted = str(genre).split('|')
    if (len(splitted) <= ind):
        return 'na'
    else:
        return splitted[ind]
songs['g1'] = songs['genre_ids'].map(lambda g: genre_dev(g, 0))
songs['g2'] = songs['genre_ids'].map(lambda g: genre_dev(g, 1))
songs['g3'] = songs['genre_ids'].map(lambda g: genre_dev(g, 2))
songs = songs.drop(['genre_ids'], axis=1)

Объединим данные, не забывая, что для word2vec нужно использовать только строчки с target = 1

In [5]:
train.head()
merged_train = train.merge(songs, on='song_id').merge(songs_extra, on='song_id')
merged_train = merged_train[merged_train['target'] == 1]

Выберем только популярные песни

In [6]:
from collections import defaultdict
s = defaultdict(int)
def add(song):
    s[song] += 1
merged_train['song_id'].map(lambda song : add(song))
ans = set()
for x in s:
    if (s[x] > 20) :
        ans.add(x)

In [7]:
merged_train2 = merged_train.loc[merged_train.song_id.isin(ans)]
merged_train4 = train.merge(songs, on='song_id').merge(songs_extra, on='song_id')
merged_train4 = merged_train4.loc[merged_train4.song_id.isin(ans)]

Наконец, необходимо сгруппировать песни для каждого пользователя, чтобы передать их Word2vec https://habr.com/ru/post/446530/

In [8]:
word_to_vec = []
grouped = merged_train2.groupby('msno')
for elem in grouped.groups:
    one_vec = grouped.get_group(elem)
    #print(one_vec['song_id'].values)
    arr = []
    for el in one_vec['song_id'].values:
        arr.append(el)
    word_to_vec.append(arr)

In [9]:
from gensim.models import Word2Vec
model = Word2Vec(min_count=1)
model.build_vocab(word_to_vec)
model.train(word_to_vec, total_examples=model.corpus_count, epochs=100)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


(309627151, 310377200)

После получения данных о песнях получим данные пользователя (сложим понравившиеся песни, чтобы получить его представление Word2Vec). Затем вернемся к изначальной таблички, чтобы добавить столбик, получающийся как скалярное произведение векторов человека на песню.

In [10]:
word_to_vec_dict = dict()
grouped = merged_train2.groupby('msno')
for elem in grouped.groups:
    one_vec = grouped.get_group(elem)
    #print(one_vec['song_id'].values)
    ans = np.zeros(100)
    amount = 0
    for el in one_vec['song_id'].values:
        ans += model.wv[el]
        amount += 1
    word_to_vec_dict[elem] = ans / amount
word_to_vec_dict

{'++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=': array([ 0.41457181, -0.25408259,  0.10137851, -0.20988634,  0.32894169,
         0.04821001,  0.22154036,  0.23679163,  0.0713147 , -0.22150454,
        -0.19496208,  0.43243789, -0.05034121, -0.19382043,  0.02842024,
         0.12347084, -0.0829137 ,  0.41547916, -0.14261952,  0.30054705,
        -0.0031075 ,  0.10622309, -0.11125165,  0.11312003,  0.19652776,
         0.16560414,  0.10637899, -0.06179426,  0.1302437 , -0.19959589,
        -0.36351043,  0.32594162, -0.03013974,  0.13828085,  0.04467831,
         0.00656439, -0.0839867 ,  0.4239042 , -0.37804991, -0.09628417,
         0.08427141, -0.20814506, -0.15281518, -0.35653681, -0.34405429,
        -0.07686281, -0.18048761,  0.11222545,  0.1271932 , -0.28834519,
        -0.51379742,  0.19143537, -0.03180728,  0.24773665,  0.14227229,
        -0.24346329, -0.26609085,  0.05258887,  0.30380916,  0.47187285,
        -0.49166654, -0.08016388,  0.16085232, -0.30096652,  0.35233396,
   

In [11]:
cosine_dict = dict()
merged_train3 = merged_train4.copy()
def make_word2vec(x):
    try:
        return np.cosine_similarity(word_to_vec_dict[x.msno], model.wv[x.song_id])
    except:
        return -1
merged_train3['word2vec'] = merged_train3.apply(lambda x : make_word2vec(x), axis=1)

In [12]:
merged_train3

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,artist_name,composer,lyricist,language,g1,g2,g3,name,isrc,word2vec
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471,Bastille,Dan Smith| Mark Crew,Unknown,52.0,359,na,na,Good Grief,GBUM71602854,2.526022
1,e5Ezre9HPuPos+CXQXtmo32E/hHIZTMmo6jG3yRf6UA=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-playlist,1,206471,Bastille,Dan Smith| Mark Crew,Unknown,52.0,359,na,na,Good Grief,GBUM71602854,-0.674530
2,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0,206471,Bastille,Dan Smith| Mark Crew,Unknown,52.0,359,na,na,Good Grief,GBUM71602854,-1.503070
3,sSexP400TJOZRhx3JB+0s9cqrCnqrlV51B9njoKR1II=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0,206471,Bastille,Dan Smith| Mark Crew,Unknown,52.0,359,na,na,Good Grief,GBUM71602854,3.602732
4,hKdGiUKHVqKkXGHLrc+EzdSW6q0ERAJ2Cs7/L1N0Ae4=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,NaN,online-playlist,0,206471,Bastille,Dan Smith| Mark Crew,Unknown,52.0,359,na,na,Good Grief,GBUM71602854,-0.267753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7365086,GiwOY8l/lt/hovRLP3bbB/J1JSDUUvA6MwTbYTfnl9U=,XXATx1PMAfjmWUqCb8rYCWrVvo3QEdjILY7XcqJw9zM=,my library,Local playlist more,local-library,1,214177,Gallant x Tablo x Eric Nam,Unknown,Unknown,52.0,139,na,na,Cave Me In,USWB11602567,18.266872
7365087,JyxQaIHWJdHAi5+FSAoetgsHZ72jpdLtuSWNa8RJWOo=,XXATx1PMAfjmWUqCb8rYCWrVvo3QEdjILY7XcqJw9zM=,discover,Online playlist more,online-playlist,1,214177,Gallant x Tablo x Eric Nam,Unknown,Unknown,52.0,139,na,na,Cave Me In,USWB11602567,3.564428
7365088,LDAkeSKc+BjZvmkdCSBcA0mW16D/AlaA3OQijsybtYk=,XXATx1PMAfjmWUqCb8rYCWrVvo3QEdjILY7XcqJw9zM=,discover,Discover Feature,online-playlist,0,214177,Gallant x Tablo x Eric Nam,Unknown,Unknown,52.0,139,na,na,Cave Me In,USWB11602567,-3.730314
7365089,sOOABBEHWMUhHFP15eAKkLnf6tYNRzi4iuFv1kgrDig=,XXATx1PMAfjmWUqCb8rYCWrVvo3QEdjILY7XcqJw9zM=,discover,Online playlist more,online-playlist,1,214177,Gallant x Tablo x Eric Nam,Unknown,Unknown,52.0,139,na,na,Cave Me In,USWB11602567,3.877042


Далее два раза используем катбуст (сначала используя все фичи, а затем только имя песни, человека и word2vec)

In [13]:
merged_train6 = merged_train3[['msno', 'song_id', 'word2vec', 'target']]

In [14]:
merged_train5 = merged_train3.reset_index(drop=True)
merged_train5['source_system_tab'] = merged_train5['source_system_tab'].fillna('Unknown')
merged_train5['source_screen_name'] = merged_train5['source_screen_name'].fillna('Unknown')
merged_train5['source_type'] = merged_train5['source_type'].fillna('Unknown')
merged_train5['language'] = merged_train5['language'].map(lambda x : str(x))
Y = merged_train5['target']
X = merged_train5.drop(['target'], axis=1)

Периодически ловил проблему CatBoostError: bad allocation, так и не понял как ее решить, приходилось обучать несколько дней, чтобы добиться результата

In [15]:
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import cross_val_score, cross_validate

paramparam = ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'artist_name', 'composer', 'lyricist', 'language', 'g1', 'g2', 'g3', 'name', 'isrc']

cv_dataset = Pool(data=X,
                  label=Y,
                  cat_features = paramparam)

model = CatBoostRegressor(task_type='CPU', loss_function='RMSE', iterations=400)
model.fit(cv_dataset)
#model = CatBoostClassifier(iterations=1000, learning_rate=1e-5, depth=4, cat_features = ["msno", "song_id", "source_system_tab", "source_screen_name", "source_type", "gender"])
#model.fit(X, Y)

params = {"iterations": 400,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          "custom_metric": 'AUC'}


scores = cv(cv_dataset,
            params,
            fold_count=4,
            shuffle=True, 
            plot=True)

scores

Learning rate set to 0.5
0:	learn: 0.4769434	total: 10.6s	remaining: 17m 34s
1:	learn: 0.4564847	total: 14.8s	remaining: 12m 7s
2:	learn: 0.4493946	total: 20s	remaining: 10m 46s
3:	learn: 0.4462369	total: 23.2s	remaining: 9m 17s
4:	learn: 0.4441384	total: 26.7s	remaining: 8m 27s
5:	learn: 0.4418494	total: 29s	remaining: 7m 34s
6:	learn: 0.4407921	total: 31.4s	remaining: 6m 57s
7:	learn: 0.4400565	total: 34s	remaining: 6m 31s
8:	learn: 0.4395451	total: 36.5s	remaining: 6m 8s
9:	learn: 0.4391075	total: 38.8s	remaining: 5m 49s
10:	learn: 0.4388344	total: 41.2s	remaining: 5m 33s
11:	learn: 0.4385849	total: 43.5s	remaining: 5m 19s
12:	learn: 0.4383167	total: 45.7s	remaining: 5m 6s
13:	learn: 0.4380957	total: 48.1s	remaining: 4m 55s
14:	learn: 0.4378787	total: 50.5s	remaining: 4m 45s
15:	learn: 0.4377287	total: 53s	remaining: 4m 37s
16:	learn: 0.4376021	total: 55.5s	remaining: 4m 31s
17:	learn: 0.4375043	total: 57.9s	remaining: 4m 23s
18:	learn: 0.4373523	total: 1m	remaining: 4m 16s
19:	lear

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std,test-AUC-mean,test-AUC-std
0,0,0.719656,0.000193,0.719655,0.000061,0.669507,0.000964
1,1,0.707498,0.000188,0.707497,0.000058,0.672573,0.001174
2,2,0.695864,0.000183,0.695864,0.000055,0.674476,0.000625
3,3,0.684735,0.000178,0.684735,0.000052,0.674488,0.000632
4,4,0.674098,0.000174,0.674097,0.000049,0.674947,0.000932
...,...,...,...,...,...,...,...
95,95,0.452303,0.000214,0.453767,0.000057,0.751378,0.000718
96,96,0.452077,0.000215,0.453555,0.000058,0.751647,0.000734
97,97,0.451859,0.000214,0.453351,0.000059,0.751881,0.000718
98,98,0.451646,0.000217,0.453153,0.000058,0.752137,0.000714


In [16]:
Y = merged_train6['target']
X = merged_train6.drop(['target'], axis=1)

from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import cross_val_score, cross_validate

paramparam = ['msno', 'song_id']

cv_dataset = Pool(data=X,
                  label=Y,
                  cat_features = paramparam)

model = CatBoostRegressor(task_type='CPU', loss_function='RMSE', iterations=400)
model.fit(cv_dataset)
#model = CatBoostClassifier(iterations=1000, learning_rate=1e-5, depth=4, cat_features = ["msno", "song_id", "source_system_tab", "source_screen_name", "source_type", "gender"])
#model.fit(X, Y)

params = {"iterations": 400,
          "depth": 2,
          "loss_function": "RMSE",
          "verbose": False,
          "custom_metric": 'AUC'}


scores1 = cv(cv_dataset,
            params,
            fold_count=4,
            shuffle=True, 
            plot=True)

scores1

Learning rate set to 0.5
0:	learn: 0.4681993	total: 1.36s	remaining: 2m 14s
1:	learn: 0.4582095	total: 2.52s	remaining: 2m 3s
2:	learn: 0.4548700	total: 3.67s	remaining: 1m 58s
3:	learn: 0.4519173	total: 5.03s	remaining: 2m
4:	learn: 0.4508465	total: 6.1s	remaining: 1m 55s
5:	learn: 0.4496517	total: 7.22s	remaining: 1m 53s
6:	learn: 0.4486470	total: 8.62s	remaining: 1m 54s
7:	learn: 0.4480696	total: 9.65s	remaining: 1m 50s
8:	learn: 0.4477430	total: 10.5s	remaining: 1m 46s
9:	learn: 0.4475407	total: 11.4s	remaining: 1m 42s
10:	learn: 0.4473146	total: 12.3s	remaining: 1m 39s
11:	learn: 0.4471799	total: 13.2s	remaining: 1m 37s
12:	learn: 0.4470799	total: 14.1s	remaining: 1m 34s
13:	learn: 0.4469916	total: 15s	remaining: 1m 32s
14:	learn: 0.4469244	total: 15.9s	remaining: 1m 30s
15:	learn: 0.4468226	total: 16.8s	remaining: 1m 28s
16:	learn: 0.4467471	total: 17.7s	remaining: 1m 26s
17:	learn: 0.4466868	total: 18.6s	remaining: 1m 24s
18:	learn: 0.4466523	total: 19.4s	remaining: 1m 22s
19:	l

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std,test-AUC-mean,test-AUC-std
0,0,0.720139,0.000194,0.720138,0.000061,0.620924,0.000656
1,1,0.708463,0.000190,0.708463,0.000057,0.627353,0.000505
2,2,0.697299,0.000186,0.697299,0.000055,0.627850,0.000553
3,3,0.686624,0.000181,0.686624,0.000054,0.629570,0.001062
4,4,0.676427,0.000180,0.676427,0.000048,0.630799,0.000685
...,...,...,...,...,...,...,...
95,95,0.457982,0.000100,0.460139,0.000063,0.740051,0.000268
96,96,0.457768,0.000098,0.459940,0.000065,0.740167,0.000279
97,97,0.457558,0.000103,0.459746,0.000063,0.740283,0.000301
98,98,0.457356,0.000105,0.459561,0.000062,0.740399,0.000308
